## 隐含波动率计算
### 牛顿迭代法

In [ ]:
#期权的隐含波动率
#牛顿迭代法求解隐含波动率
#call的计算
def impvol_call_Newton(C,S,K,r,T,sigma0 = 0.2):
    def call_BS(S,K,sigma,r,T):
        import numpy as np
        from scipy.stats import norm
        d1 = (np.log(S/K)+(r+sigma**2/2)*T)/(sigma*np.sqrt(T))
        d2 = d1 - sigma*np.sqrt(T)
        
        return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
        
    diff = C - call_BS(S, K, sigma0, r, T)
    i = 0.0001
    while abs(diff) > 0.0001:
        diff = C - call_BS(S, K, sigma0, r, T)
        if diff > 0:
            sigma0 += i
        else:
            sigma0 -= i
    
    return sigma0

#put的计算
def impvol_put_Newton(P,S,K,r,T,sigma0 = 0.2):
    def put_BS(S,K,sigma,r,T):
        import numpy as np
        from scipy.stats import norm
        d1 = (np.log(S/K)+(r+sigma**2/2)*T)/(sigma*np.sqrt(T))
        d2 = d1 - sigma*np.sqrt(T)
        return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)
        
    diff = P - put_BS(S, K, sigma0, r, T)
    i = 0.0001
    while abs(diff) > 0.0001:
        diff = P - put_BS(S, K, sigma0, r, T)
        if diff > 0:
            sigma0 += i
        else:
            sigma0 -= i
    
    return sigma0

impvol_call = impvol_call_Newton(C=0.1566,S=5.29,K=6,r=0.04,T=0.5,sigma0 = 0.2)
impvol_put = impvol_put_Newton(P=0.7503,S=5.29,K=6,r=0.04,T=0.5,sigma0 = 0.2)
print('牛顿法计算的看涨期权隐含波动率:',round(impvol_call,4))
print('牛顿法计算的看跌期权隐含波动率:',round(impvol_put,4))


### 二分查找法

In [ ]:
#二分法求解隐含波动率
#call的隐含波动率
def impvol_call_Binary(C,S,K,r,T,sigma_min = 0.001,sigma_max = 1.00):
    #call的计算
    def call_BS(S,K,sigma,r,T):
        import numpy as np
        from scipy.stats import norm
        d1 = (np.log(S/K)+(r+sigma**2/2)*T)/(sigma*np.sqrt(T))
        d2 = d1 - sigma*np.sqrt(T)
        
        return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
        
    sigma_mid = (sigma_min + sigma_max)/2
    call_min = call_BS(S, K, sigma_min, r, T)
    call_max = call_BS(S, K, sigma_max, r, T)
    call_mid = call_BS(S, K, sigma_mid, r, T)
    diff = C - call_mid
    if C < call_min:
        print('Error:请重新输入更小的sigma_min')
    if C > call_max:
        print('Error:请重新输入更大的sigma_max')
    while abs(diff) > 0.000001:
        diff = C - call_BS(S, K, sigma_mid, r, T)
        sigma_mid = (sigma_min + sigma_max)/2
        call_mid = call_BS(S, K, sigma_mid, r, T)
        if C > call_mid:
            sigma_min = sigma_mid
        else:
            sigma_max = sigma_mid
    
    return sigma_mid    

#put的隐含波动率
def impvol_put_Binary(P,S,K,r,T,sigma_min = 0.001,sigma_max = 1.00):
    #put的计算
    def put_BS(S,K,sigma,r,T):
        import numpy as np
        from scipy.stats import norm
        d1 = (np.log(S/K)+(r+sigma**2/2)*T)/(sigma*np.sqrt(T))
        d2 = d1 - sigma*np.sqrt(T)
        return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)

    sigma_mid = (sigma_min + sigma_max)/2
    put_min = put_BS(S, K, sigma_min, r, T)
    put_max = put_BS(S, K, sigma_max, r, T)
    put_mid = put_BS(S, K, sigma_mid, r, T)
    diff = P - put_mid
    if P < put_min:
        print('Error:请重新输入更小的sigma_min')
    if P > put_max:
        print('Error:请重新输入更大的sigma_max')
    while abs(diff) > 0.000001:
        diff = P - put_BS(S, K, sigma_mid, r, T)
        sigma_mid = (sigma_min + sigma_max)/2
        put_mid = put_BS(S, K, sigma_mid, r, T)
        if P > put_mid:
            sigma_min = sigma_mid
        else:
            sigma_max = sigma_mid
    
    return sigma_mid  
    
impvol_call_Binary = impvol_call_Binary(C=0.1566,S=5.29,K=6,r=0.04,T=0.5,sigma_min = 0.001,sigma_max = 1)
impvol_put_Binary = impvol_put_Binary(P=0.7503,S=5.29,K=6,r=0.04,T=0.5,sigma_min = 0.001,sigma_max = 1)    

print('二分法计算的看涨期权隐含波动率:',round(impvol_call_Binary,4))
print('二分法计算的看跌期权隐含波动率:',round(impvol_put_Binary,4))


### 波动率微笑

In [ ]:
#波动率微笑
#2018/6/27到期的、不同执行价格的上证50ETF期权合约在2017/12/29日的收盘价为例
import datetime as dt

T1 = dt.datetime(2017, 12, 29) #计算隐含波动率时的日期
T2 = dt.datetime(2018, 6, 27) #期权到期日
T_delta = (T2 - T1).days/365 # 期权的剩余期限并转化成年
S0 = 2.859 # 当日的上证50ETF基金净值
#认购(call)期权收盘价
call_list = np.array([0.2841,0.2486,0.2139,0.1846,0.1586,0.1369,0.1177])
#认沽(put)期权收盘价
put_list = np.array([0.0464,0.0589,0.0750,0.0947,0.1183,0.1441,0.1756])
#期权执行价格
K_list = np.array([2.7,2.75,2.8,2.85,2.9,2.95,3.0])
shibor = 0.048823 # 计算隐含波动率当日6个月期的shibor利率

sigma_clist = np.zeros_like(call_list)
#使用牛顿法计算隐含波动率
for i in np.arange(len(call_list)):
    sigma_clist[i] = impvol_call_Newton(C=call_list[i], S=S0, K=K_list[i], r=shibor, T=T_delta,sigma0 = 0.2)

print('call的隐含波动率:',sigma_clist)


sigma_plist = np.zeros_like(put_list)
for i in np.arange(len(put_list)):
    sigma_plist[i] = impvol_put_Newton(P=put_list[i], S=S0, K=K_list[i], r=shibor, T=T_delta,sigma0 = 0.2)

print('put的隐含波动率:',sigma_plist)

plt.figure(figsize=(8,6))
plt.plot(K_list,sigma_clist,label='50ETF认购期权')
plt.plot(K_list,sigma_plist,label='50ETF认沽期权')
plt.legend()
plt.grid('True')


### 波动率斜偏

In [ ]:
#波动率偏斜
#2019/6/26到期的、不同执行价格的上证50ETF期权合约在2018/12/28日的收盘价为例
import datetime as dt

T1 = dt.datetime(2018, 12, 28) #计算隐含波动率时的日期
T2 = dt.datetime(2019, 6, 26) #期权到期日
T_delta = (T2 - T1).days/365 #期权的剩余期限并转化成年
S0 = 2.289 #当日的上证50ETF基金净值
#认购(call)期权收盘价
call_list = np.array([0.2866,0.2525,0.2189,0.1912,0.1645,0.1401,0.1191,0.0996,0.0834,0.0690,0.0566,0.0464,0.0375])
#认沽(put)期权收盘价
put_list = np.array([0.0540,0.0689,0.0866,0.1061,0.1294,0.1531,0.1814,0.2122,0.2447,0.2759,0.3162,0.3562,0.3899])
#期权执行价格
K_list = np.array([2.1,2.15,2.2,2.25,2.3,2.35,2.4,2.45,2.5,2.55,2.6,2.65,2.7])
shibor = 0.03297 #计算隐含波动率当日6个月期的shibor利率

sigma_clist = np.zeros_like(call_list)
#使用牛顿法计算隐含波动率
for i in np.arange(len(call_list)):
    sigma_clist[i] = impvol_call_Newton(C=call_list[i], S=S0, K=K_list[i], r=shibor, T=T_delta,sigma0 = 0.2)

print('call的隐含波动率:',sigma_clist)


sigma_plist = np.zeros_like(put_list)
for i in np.arange(len(put_list)):
    sigma_plist[i] = impvol_put_Newton(P=put_list[i], S=S0, K=K_list[i], r=shibor, T=T_delta,sigma0 = 0.2)

print('put的隐含波动率:',sigma_plist)
    
plt.figure(figsize=(8,6))
plt.plot(K_list,sigma_clist,label='50ETF认购期权')
plt.plot(K_list,sigma_plist,label='50ETF认沽期权')
plt.legend()
plt.grid('True')
